<a href="https://colab.research.google.com/github/crypmario/btlm-on-colab/blob/main/btlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


# install packages

In [2]:
!pip install -q -U scipy
!pip install -U -q git+https://github.com/huggingface/transformers@de9255de27abfcae4a1f816b904915f0b1e23cd9
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 44.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!git clone https://github.com/timdettmers/bitsandbytes.git
%cd bitsandbytes
!CUDA_VERSION=118 make cuda11x
!python setup.py install

Cloning into 'bitsandbytes'...
remote: Enumerating objects: 4636, done.
remote: Counting objects: 100% (2289/2289), done.
remote: Compressing objects: 100% (338/338), done.
remote: Total 4636 (delta 2162), reused 1961 (delta 1951), pack-reused 2347
Receiving objects: 100% (4636/4636), 1.37 MiB | 2.58 MiB/s, done.
Resolving deltas: 100% (3238/3238), done.
/content/bitsandbytes
mkdir -p build
mkdir -p dependencies
ENVIRONMENT
CUDA_VERSION: 118
NVCC path: /usr/local/cuda/bin/nvcc
GPP path: /usr/bin/g++ VERSION: g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
CUDA_HOME: /usr/local/cuda
CONDA_PREFIX: 
PATH: /opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
LD_LIBRARY_PATH: /usr/lib64-nvidia
/usr/local/cuda/bin/nvcc -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -Xcompiler '-fPIC' --use_fast_math -Xptxas=-v -dc /content/bitsandbytes/

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load tokenizer and the model

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    'cerebras/btlm-3b-8k-base',
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map='auto'
)

# check we are in 4bit
model.transformer.h[3].attn.c_attn

Conv1D()

In [7]:
tokenizer = AutoTokenizer.from_pretrained("cerebras/btlm-3b-8k-base")

# Text prompting

## Tokenize prompt

In [8]:
# Set the prompt for generating text
prompt = "Albert Einstein was known for "

# Tokenize the prompt and convert to PyTorch tensors
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

## Generate response

In [12]:
# Generate text using the model
outputs = model.generate(
    **inputs,
    num_beams=5,
    max_new_tokens=50,
    early_stopping=True,
    no_repeat_ngram_size=2
)

# Convert the generated token IDs back to text
generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Print the generated text
print(generated_text[0])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1448: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Albert Einstein was known for 
his theory of relativity and his work on the photoelectric effect.
He was awarded the 1921 Nobel Prize in Physics "for his services
to theoretical physics, and especially for his discovery of the law
which unifies in a single equation the
